# DNN with `grblas`

- Problem from: http://graphchallenge.mit.edu/challenges
- Data from: http://graphchallenge.mit.edu/data-sets
- Based on: https://github.com/GraphBLAS/LAGraph/blob/666b4a1fd0c7c27395dc912ecfca2cfe108dd3e4/Experimental/Algorithm/LAGraph_dnn.c

In [ ]:
import grblas as gb
import numpy as np
import pandas as pd
from grblas import Vector, Matrix, replace

In [ ]:
# Indices in files are 1-based, so we always subtract one
nlayers = 120
nneurons = 1024
nfeatures = 60000
neural_net_bias = -0.3
ymax = 32.0
dtype = gb.dtypes.FP64  # FP32 should also work (and be faster)
W = []
Bias = []

In [ ]:
for i in range(1, nlayers + 1):
    df = pd.read_csv(f'Matrices/neuron{nneurons}/n{nneurons}-l{i}.tsv', delimiter='\t', header=None)
    W.append(Matrix.from_values(
        df[0].values - 1, df[1].values - 1, df[2].values,
        nrows=nneurons, ncols=nneurons, dtype=dtype,
    ))

In [ ]:
a = np.arange(nneurons)
for i in range(nlayers):
    Bias.append(Matrix.from_values(
        a, a, np.repeat(neural_net_bias, nneurons),
        nrows=nneurons, ncols=nneurons, dtype=dtype,
    ))

In [ ]:
df = pd.read_csv(f'Matrices/neuron{nneurons}/neuron{nneurons}-l{nlayers}-categories.tsv', header=None)
true_categories = Vector.from_values(df[0].values - 1, np.repeat(True, len(df)), size=nfeatures)

In [ ]:
df = pd.read_csv(f'Matrices/neuron{nneurons}/sparse-images-{nneurons}.tsv', delimiter='\t', header=None)
Y0 = Matrix.from_values(
    df[0].values - 1, df[1].values - 1, df[2].values,
    nrows=nfeatures, ncols=nneurons, dtype=dtype,
)

In [ ]:
def DNN(W, Bias, Y0):
    dtype = Y0.dtype
    nlayers = len(W)
    nfeatures = Y0.nrows
    nneurons = Y0.ncols

    plus_times = gb.semiring.plus_times[dtype]
    plus_plus = gb.semiring.plus_plus[dtype]
    gt = gb.binary.gt[dtype]
    fmin = gb.binary.min
    identity = gb.unary.identity[dtype]

    Y = Matrix.new(dtype, nrows=nfeatures, ncols=nneurons)
    M = Matrix.new(bool, nrows=nfeatures, ncols=nneurons)
    for layer in range(nlayers):
        if layer == 0:
            Y << plus_times(Y0 @ W[layer])
        else:
            Y << plus_times(Y @ W[layer])
        Y << plus_plus(Y @ Bias[layer])

        # select: Y << Y[gt(Y, 0)]
        M << gt(Y, 0)
        Y(M.V, replace) << identity(Y)

        Y << fmin(Y, ymax)
    return Y

In [ ]:
%%time
Y = DNN(W, Bias, Y0)
categories = Y.reduce_rows().new(dtype=bool)
assert categories.isequal(true_categories)

In [ ]:
# What happens if we create new matrices?
def DNN2(W, Bias, Y0):
    dtype = Y0.dtype
    nlayers = len(W)
    nfeatures = Y0.nrows
    nneurons = Y0.ncols

    plus_times = gb.semiring.plus_times[dtype]
    plus_plus = gb.semiring.plus_plus[dtype]
    gt = gb.binary.gt[dtype]
    fmin = gb.binary.min
    identity = gb.unary.identity[dtype]

    Y = Y0
    for layer in range(nlayers):
        Y = plus_times(Y @ W[layer]).new()
        Y = plus_plus(Y @ Bias[layer]).new()

        # select: Y << Y[gt(Y, 0)]
        M = gt(Y, 0).new()
        Y(M.V, replace) << identity(Y)

        Y = fmin(Y, ymax).new()
    return Y

In [ ]:
%%time
Y = DNN(W, Bias, Y0)
categories = Y.reduce_rows().new(dtype=bool)
assert categories.isequal(true_categories)